https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Code ini merupakan tutorial quickstart dari official web PyTorch.

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Dataset FashionMNIST sudah ada sebagai bagian dari library PyTorch. Hasil
download terdiri dari 2 macam dataset: dataset&label untuk training dan
dataset&label untuk testing.

In [6]:
# jika belum pernah di-download, download ke folder data
# untuk argumen train=True, gunakan dataset untuk training (train-images-idx3-ubyte)
# transform=ToTensor() mengubah PIL Images menjadi format tensor
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# untuk argumen train=False, gunakan dataset untuk testing (t10k-images-idx3-ubyte)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Dataloader merupakan sebuah iterable yang support automatic batching, sampling,
shuffling, dan multiprocess data loading.

Jika batch_size=64 dan training_data=60000,
maka train_dataloader = training_data / batch_size = 937.5 = 938.

https://www.kaggle.com/datasets/zalando-research/fashionmnist

Ukuran satu data FashionMNIST adalah 28 pixels x 28 pixels.

In [19]:
batch_size = 64

# create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print("training_data:", len(training_data))
print("test_data:", len(test_data))
print("train_dataloader:", len(train_dataloader))
print("test_dataloader:", len(test_dataloader))

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

training_data: 60000
test_data: 10000
train_dataloader: 938
test_dataloader: 157
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Jika ada GPU Nvidia, gunakan cuda. 

Jika ada Nvidia Multi-Process Service, gunakan mps.
https://pytorch.org/serve/nvidia_mps.html

Jika tidak ada satupun, gunakan cpu.

In [20]:
device = (
    "cuda" if torch.cuda.is_available()
        else "mps" if torch.backends.mps.is_available()
        else "cpu"
)

print(f"Using {device} device")

Using cpu device


Arsitektur neural network.

28*28 sesuai dengan ukuran 1 example data FashionMNIST.

Pada layer terakhir akan menghasilkan 10 output dengan probabilitas
masing-masing. 

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # computer prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------
loss: 2.288001 [   64/60000]
loss: 2.286509 [ 6464/60000]
loss: 2.267801 [12864/60000]
loss: 2.266342 [19264/60000]
loss: 2.251616 [25664/60000]
loss: 2.221605 [32064/60000]
loss: 2.232651 [38464/60000]
loss: 2.198821 [44864/60000]
loss: 2.193788 [51264/60000]
loss: 2.167642 [57664/60000]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.159934

Epoch 2
------------
loss: 2.163356 [   64/60000]
loss: 2.160763 [ 6464/60000]
loss: 2.103959 [12864/60000]
loss: 2.115252 [19264/60000]
loss: 2.073487 [25664/60000]
loss: 2.018971 [32064/60000]
loss: 2.040313 [38464/60000]
loss: 1.968054 [44864/60000]
loss: 1.969828 [51264/60000]
loss: 1.898450 [57664/60000]
Test Error: 
 Accuracy: 59.1%, Avg loss: 1.895082

Epoch 3
------------
loss: 1.925331 [   64/60000]
loss: 1.895560 [ 6464/60000]
loss: 1.784524 [12864/60000]
loss: 1.811363 [19264/60000]
loss: 1.710054 [25664/60000]
loss: 1.669132 [32064/60000]
loss: 1.680232 [38464/60000]
loss: 1.588543 [44864/60000]
loss: 1.610046 [51264/6

In [13]:
torch.save(model.state_dict(), "quickstart.pth")
print("Saved PyTorch Model State to quickstart.pth")

Saved PyTorch Model State to quickstart.pth


In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("quickstart.pth"))

<All keys matched successfully>

test_data adalah dataset testing dari FashionMNIST. example terakhirnya adalah
test_data[9999].

Jika akurasi rendah, kemungkinan prediksi salah cukup besar.

In [41]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()

# urutan example di dalam testing dataset. dari 0 hingga 9999.
NUMBER = 9996

x, y = test_data[NUMBER][0], test_data[NUMBER][1]

print("x:", x.shape)
print("y:", y)

with torch.no_grad():
    # convert 
    x = x.to(device)
    pred = model(x)
    print("pred:", pred)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

x: torch.Size([1, 28, 28])
y: 1
pred: tensor([[ 1.0367,  2.5055, -0.1181,  1.7971,  0.2553, -1.3577,  0.4084, -1.7467,
         -1.3247, -1.6003]])
Predicted: "Trouser", Actual: "Trouser"
